In [2]:
nlp_model = 'lda'

from experiments._60_nlpmodels.fit_social_nlp_models import *

from sklearn.metrics import f1_score

f1s_social = load_nlp_selected_users()

nlp_uids = [x[0] for x in f1s_social]

uid_pos = {uid: pos for pos, uid in enumerate(nlp_uids)}

In [3]:
# def scale(X_train, X_test):
#     train_size = X_train.shape[0]
#     X = np.concatenate((X_train, X_test))
#     scaler = StandardScaler()
#     scaler.fit(X)
#     X = scaler.transform(X)
#     X_train = X[:train_size,:]
#     X_test = X[train_size:,:]

#     return X_train, X_test, scaler

# f1s_combined = {}

# for n_topics in [10, 16, 20]:
#     f1s_n = {'train': [], 'valid': [], 'test': []}
#     nlp_feats = pd.read_pickle('alltweets_es_%s%d.pickle' % (nlp_model, n_topics))

#     for uid, f1 in f1s_social:
#         uid = int(uid)
#         clf = load_model(uid, 'svc' + nlp_model, n_topics=n_topics, subfolder='social_nlp')

#         # Train classifiers only if not created already
#         print "==============================" 
#         print "Processing %d ( f1 %.2f %%)" % (uid, 100 * f1)

#         X_train, X_valid, X_test, y_train, y_valid, y_test = load_dataframe(uid)

#         X_train_nlp = nlp_feats.loc[X_train.index]
#         X_valid_nlp = nlp_feats.loc[X_valid.index]
#         X_test_nlp = nlp_feats.loc[X_test.index]

#         X_train_combined = np.hstack((X_train, X_train_nlp))
#         X_valid_combined = np.hstack((X_valid, X_valid_nlp))
#         X_test_combined = np.hstack((X_test, X_test_nlp))

#         X_train_combined, X_valid_combined, scaler = scale(X_train_combined, X_valid_combined)
#         # Aplicar mismo scaling a X_test
#         X_test_combined = scaler.transform(X_test_combined)
        
#         y_true, y_pred = y_train, clf.predict(X_train_combined)
#         f1s_n['train'].append(f1_score(y_true, y_pred))
        
#         y_true, y_pred = y_valid, clf.predict(X_valid_combined)
#         f1s_n['valid'].append(f1_score(y_true, y_pred))
        
#         y_true, y_pred = y_test, clf.predict(X_test_combined)
#         f1s_n['test'].append(f1_score(y_true, y_pred))
        
#     f1s_combined[n_topics] = f1s_n

In [4]:
# with open('f1s_combined_%s.json' % nlp_model, 'w') as f:
#     json.dump(f1s_combined, f)

In [5]:
with open('f1s_combined_%s.json' % nlp_model) as f:
    f1s_combined = json.load(f)

# Comparación sobre set de prueba

In [6]:
from bokeh.charts import Line, show, output_notebook

output_notebook()

Loading BokehJS ...

In [7]:
f1s_test_social = json.load(open('../_1_one_user_learn_neighbours/f1s_testv_svc.json'))
f1s_test_social = {u: f for u, f in f1s_test_social.items() if u in nlp_uids}

In [8]:
sorted_uids = [u for u, f in sorted(f1s_test_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

In [9]:
f1s_10 = np.array([f1s_combined['10']['test'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_20 = np.array([f1s_combined['20']['test'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_s = np.array([f1s_test_social[sorted_uids[i]] for i in indices])

In [10]:
# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+lda10": f1s_10,
    "social+lda20": f1s_20
}

labels = 'social social+lda10 social+lda20'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuarios (ordenados por f1 del modelo social en datos de evaluación)',
            ylabel='puntaje f1 sobre datos de evaluación', legend=True)

# output_file("line_single.html", title="line_single.py example")

show(line)

In [11]:
sum(f1s_10 > f1s_s)

4

In [12]:
sum(f1s_20 > f1s_s)

5

In [13]:
(np.mean(f1s_10), np.mean(f1s_20))

(0.72020639395793973, 0.70696678542391889)

In [14]:
for f in [f1s_10, f1s_20]:
    better = f > f1s_s
    mean_improvement = np.mean(f[better] - f1s_s[better])
    print mean_improvement

0.0275975996318
0.0381733668966


# Comparación en set de entrenamiento

In [15]:
f1s_train_social = json.load(open('../_1_one_user_learn_neighbours/f1s_train_svc.json'))

In [16]:
f1s_train_social = {u: f for u, f in f1s_train_social.items() if u in nlp_uids}

In [17]:
sorted_uids = [u for u, f in sorted(f1s_train_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

In [18]:
f1s_10 = np.array([f1s_combined['10']['train'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_20 = np.array([f1s_combined['20']['train'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_s = np.array([f1s_train_social[sorted_uids[i]] for i in indices])

In [19]:
# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+lda10": f1s_10,
    "social+lda20": f1s_20
}

labels = 'social social+lda10 social+lda20'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            ylabel='puntaje f1 sobre datos de entrenamiento',
            xlabel='usuarios (ordenados por f1 del modelo social en datos de entrenamiento)',
            legend=True)

# output_file("line_single.html", title="line_single.py example")

show(line)

In [20]:
sum(f1s_10 > f1s_s)

31

In [21]:
sum(f1s_20 > f1s_s)

31

In [22]:
(np.mean(f1s_10), np.mean(f1s_20))

(0.87822639983730644, 0.9110657339143291)

In [23]:
for f in [f1s_10, f1s_20]:
    better = f > f1s_s
    mean_improvement = np.mean(f[better] - f1s_s[better])
    print mean_improvement

0.166084491573
0.199839841618
